    Variables Description:
- C1 Number of distinct terminals in the grid
- C2 Number of distinct terminals in roaming in the grid
- C3 Number of distinct terminals that remained in the grid
- C4 Number of distinct terminals that remained in the grid in roaming
- C5 Number of distinct terminal entries in the grid
- C6 Number of distinct terminal exits from the grid
- C7 Number of distinct terminal entries in roaming in the grid
- C8 Number of distinct terminal exits in roaming from the grid
- C9 Number of distinct terminals with active data connection in the grid
- C10 Number of distinct terminals with active data connection in roaming in the grid
- C11 Number of voice calls originating from the grid
- C12 Number of entries in Lisbon through the 11 main road axes
- C13 Number of exits from Lisbon through the 11 main road axes
- D1 Top 10 countries of origin for roaming terminal equipment
- E1 Number of voice calls that terminated in the grid
- E2 Average downstream rate in the grid within the grid
- E3 Average upstream rate in the grid
- E4 Peak downstream rate in the grid
- E5 Peak upstream rate in the grid
- E6 Top 10 Applications (Text with names separated by ;)
- E7 Minimum duration of stay within the grid
- E8 Average duration of stay within the grid
- E9 Maximum duration of stay within the grid
- E10 Number of devices that participate in the connection sharing within the grid

In [ ]:
import pandas as pd
import numpy as np
import glob

csv_files = glob.glob('data/DataSetSamples/sample_April2022.csv')

dfs = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file, dtype={'E6': 'string'})
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)


In [ ]:
print(df.shape)
df.head()

In [ ]:
print(df.info(show_counts=True))

Columns with a NaN values in the dataset:
- **Unnamed: 0.1** - have only index values in float type, so we can drop it
- **Unnamed: 0** - same as above, but index values are in int type
- **extract_year, extract_month, extract_day** - we have **Datetime**, so we can drop it
- **D1** - include roaming information in the grid, so NaN values also represent some information, which can be used later
- **E6** - include most used apps in the grid, so NaN values can also represent some information, but it needs future investigation
- **extract_year_2, extract_month_3, extract_day_4** - we have **DateTime** column, so we can drop it

In [ ]:
drop_columns = ['Unnamed: 0', 'Unnamed: 0.1', 'extract_year', 'extract_month', 'extract_day', 'extract_year_2', 'extract_month_3', 'extract_day_4']
df.drop(drop_columns, axis=1, inplace=True)
df.head()

    Main road to consideration:
- Ponte Vasco da Gama:
3651/3629
- A36 (Túnel do Grilo)
3563/3564/3565
- IC16
3319/3320
- N117 (Cabos Ávila)
933/934/990
- Marginal
75/76
- IC2 (Sacavém)
3736/3728
- A1
3524/3564
- Calçada de Carriche
3460/3500/3501
- IC19
1758/1759/1699
- A5
698/757/758
- Ponte 25 Abril
102/103

In [ ]:
def assign_street_name(grid_id):
    for street, grid_ids in street_names.items():
        if grid_id in grid_ids:
            print(f"grid_id: {grid_id}, street: {street}")
            return street
    return None



csv_files_1 = glob.glob('data/June2023.csv')
#  Create a list of road names from main roads in Lisbon with grid IDs
street_names = {
    "Ponte Vasco da Gama": [3651, 3629],
    "A36": [3563, 3564, 3565],
    "IC16": [3319, 3320],
    "N117": [933, 934, 990],
    "Marginal": [75, 76],
    "IC2 (Sacavém)": [3736, 3728],
    "A1": [3524, 3564],
    "Calcada De Carriche": [3460, 3500, 3501],
    "IC19": [1758, 1759, 1699],
    "A5": [689, 757, 758],
    "Ponte 25 Abril 102/103": [102, 103],
}
dfs = []

for csv_file in csv_files_1:
    df_road = pd.read_csv(csv_file, dtype={'E6': 'string'})
    dfs.append(df_road)

df_road = pd.concat(dfs, ignore_index=True)
df_road['Street_Name'] = df_road['Grid_ID'].apply(assign_street_name)
# ponte_vasco_da_gama = df_road.loc[(df_road['Grid_ID'] >= 3629) & (df_road['Grid_ID'] <= 3651)]
# print(main_roads)